In [3]:
import typing
import uuid
from dataclasses import dataclass, field


FilePath: typing.TypeAlias = str
StreamOfChars: typing.TypeAlias = typing.Iterator[str]


class OpenStream(typing.Protocol):
    def __call__(self, file: FilePath) -> StreamOfChars: ...


def stream_opener(file: FilePath) -> StreamOfChars:
    with open(file) as file_handler:
        while character := file_handler.read(1):
           yield character


read_char: OpenStream = stream_opener


@dataclass
class Point:
    lat: int
    long: int
    height: int = 0
    trailhead: bool = False
    peaks: list["Point"] = field(default_factory=list)

    def add_peak(self, peak: "Point"):
        if self.trailhead is False:
            self.trailhead = True
        if peak not in self.peaks:
            self.peaks.append(peak)


@dataclass
class Row:
    points: list[Point] = field(default_factory=list)

    def add_point(self, point: Point):
        self.points.append(point)


@dataclass
class Map:
    rows: list[Row] = field(default_factory=list)
    trails: list["Trail"] = field(default_factory=list)
    point_getters: list[typing.Callable] = field(default_factory=list)

    def __post_init__(self):
        self.point_getters += [
            self.get_point_up, self.get_point_right, self.get_point_down, self.get_point_left
        ]

    def load_points(self, file_opener: OpenStream, file_path: str):
        row = Row()
        for character in file_opener(file_path):
            if character == "\n":
                self.add_row(row)
                row = Row()
                continue
            if character == ".":
                row.add_point(Point(lat=len(self.rows), long=len(row.points), height=-1))
                continue
            height = int(character)
            row.add_point(Point(lat=len(self.rows), long=len(row.points), height=height))

    def count_trailheads(self) -> list[Point]:
        trailheads: list[Point] = []
        for row in self.rows:
            for point in row.points:
                if point.trailhead is True:
                    trailheads.append(point)
        return trailheads

    def count_score(self) -> int:
        score = 0
        for row in self.rows:
            for point in row.points:
                if point.trailhead is True:
                    score += len(point.peaks)
        return score

    def add_row(self, row: Row):
        self.rows.append(row)

    def add_trail(self, trail: "Trail"):
        self.trails.append(trail)

    def get_point_up(self, point: Point) -> Point:
        col = point.long
        row = point.lat - 1
        if row < 0:
            raise ValueError("Asked point up from: ", point, ", is out of the map")
        return self.rows[row].points[col]

    def get_point_down(self, point: Point) -> Point:
        col = point.long
        row = point.lat + 1
        if row > len(self.rows) - 1:
            raise ValueError("Asked point down from: ", point, ", is out of the map")
        return self.rows[row].points[col]

    def get_point_left(self, point: Point) -> Point:
        col = point.long - 1
        row = point.lat
        if col < 0:
            raise ValueError("Asked point left from: ", point, ", is out of the map")
        return self.rows[row].points[col]

    def get_point_right(self, point: Point) -> Point:
        col = point.long + 1
        row = point.lat
        if col > len(self.rows[row].points) - 1:
            raise ValueError("Asked point right from: ", point, ", is out of the map")
        return self.rows[row].points[col]

    def try_new_trail(self, trailhead: Point, point_getter: typing.Callable):
        trail: Trail = Trail()
        try:
            point: Point = point_getter(trailhead)
            trail.add_point(trailhead)
            trail.add_point(point)
        except ValueError as exc:
            ...
        else:
            self.add_trail(trail)    

    def find_new_trails(self):
        for row, instance in enumerate(self.rows):
            for col, _ in enumerate(instance.points):
                point: Point = self.rows[row].points[col]
                for point_getter in self.point_getters:
                    if point.height == 0:
                        self.try_new_trail(point, point_getter)

    def find_not_processed_trails(self):
        return [trail for trail in self.trails if trail.processed is False]

    def finish_trails(self):
        while trails := self.find_not_processed_trails():
            for trail in trails:
                trail.find_way(self)


@dataclass
class Trail:
    id: uuid.UUID = field(default_factory=uuid.uuid4)
    path: list[Point] = field(default_factory=list)
    processed: bool = False

    def next_point(self, trail_map: Map, point: Point) -> Point | None:
        next_points: list[Point] = []
        for point_getter in trail_map.point_getters:
            try:
                next_point: Point = point_getter(point)
                self.__validate_point(next_point)
                next_points.append(next_point)       
                if len(next_points) > 1:
                    new_trail: Trail = Trail(path=list(self.path))
                    new_trail.add_point(next_point)
                    trail_map.add_trail(new_trail)
            except ValueError as exc:
                ...
        if next_points:
            return next_points[0]

    def find_way(self, trail_map: Map):
        if self.processed is False:
            self.processed = True
        last_point: Point = self.path[-1]
        try:
            while next_point := self.next_point(trail_map, last_point):
                self.add_point(next_point)
                last_point = next_point
        except ValueError as exc:
            ...    

    def add_point(self, point: Point):
        self.__validate_point(point)
        self.path.append(point)
        self.__post_add_point(point)

    def __validate_point(self, point):
        if len(self.path) > 0:
            last_point = self.path[-1]
            if abs(last_point.lat - point.lat) + abs(last_point.long - point.long) > 1:
                raise ValueError("Point: ", point, ", is not near to: ", last_point)
            if point.height - last_point.height != 1:
                raise ValueError("Point: ", point, ", does not fit in the path: ", self.path)
        else:
            if point.height != 0:
                raise ValueError("Point: ", point, ", cannot be start of the trail")

    def __post_add_point(self, point):
        if point.height == 9:
            trailhead: Point = self.path[0]
            trailhead.add_peak(point)

In [4]:
from rich import print

file_path: FilePath = "../media/2024-day-10.input"

trail_map = Map()
trail_map.load_points(read_char, file_path)
trail_map.find_new_trails()
trail_map.finish_trails()

print(open(file_path).read())
print("Score: ", trail_map.count_score())

14567892107654348943218769016567650154541210421036
03456783298993267654309458122168743243450344323145
12567654456780154327812367433059804012769455410234
03498012349876065016901056544965418765898766708943
12345101212145076545411034545878329658981055899854
09876876705034187632110123656789421047432765988765
67878965896123298901001656743078431236598894012034
50965014387654567650012349856127340012367653213125
41234321298347656543243492347833458903458743404987
30087430178298343650156781016942167812769252985676
21196567069121243761056432679851043212890101679854
33203498451080252852347841589765654301285234521763
14512432347890161943210950432106567610106501430012
01693501036543270856102167645656788943217432567897
32789672321015389987343078938765497654998549879898
45679987410234578101256560129812321067801456734787
03478756500187665432107452121901054328982340125676
12568767891098987013898943030810167017654321010210
21079458910127698123965436945107878988901267124378
30980349821034787654876327876716901210985458095469
45671210136765693454761016329825432345671329186954
12789800345876548763876125419434501654510413277843
03543211238989439012985630308765898746701204567832
14623400141232323101234521678906567239874343236901
25710519850541014143219834567611452108965650145690
76897678769650001054301712106320143210345789036781
87678989678742112363212601235431234321276988325432
90549876349233678478004592347842389123489676710876
21632305256104569589123487656965476016512369856945
52301014107012345670149874565456365017603450747832
65490123458912396501234563432147454328214921632401
86985432167905487654341012563038901039309834521321
97876789001856778761232127678127612398712701100410
89810678012760869890103238999210543125625632234509
76701549013451987217876434785695610034534548765678
05432432174012987301987325654780123435210159854789
12980120985123673458986510783279234987346543123898
43878921976034562567603412892168765679857012010187
34565437852178901070412103601001410012768001921236
45430566543065012181543014580432321003459122876545
50121098767654327892678877698569457654219433468904
23292145678954218983019988087658768894308596567812
14587239010563007654128679112565489765107687656521
05674678323472167659436543203474321087230156785430
96983565401089898748540987654589321098543243896543
87874328992396701037621296562105465407698012565432
78765017687478632128760345673456978312789801478521
29653078596569543019656921087567889213456700329650
12532169430430156010567892193610367804765410418789
03445678321321060123456543012323458912894321001678

Score:  638